In [2]:
!pip install numpy pandas scikit-learn tensorflow deap imbalanced-learn


In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer

from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Content/KDDTest+.txt', header=None)

# Define feature columns and target column
feature_columns = data.columns[:-1]
target_column = data.columns[-1]

# Identify categorical and numerical columns
categorical_cols = data[feature_columns].select_dtypes(include=['object']).columns
numerical_cols = data[feature_columns].select_dtypes(exclude=['object']).columns

# Preprocess the features
# Use ColumnTransformer to apply different transformers to different columns
transformer = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])
X = transformer.fit_transform(data[feature_columns])

# Encode the target variable and convert to a NumPy array
y = pd.get_dummies(data[target_column]).values
# Use .values to extract the underlying NumPy array from the DataFrame

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

Mounted at /content/drive


In [4]:
!pip install deap
import random
from deap import base, creator, tools, algorithms
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
import tensorflow as tf # Import TensorFlow
# Define helper functions
def build_model(input_dim, output_dim): # Define the build_model function
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_dim=input_dim),
        tf.keras.layers.Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train):
    model = build_model(X_train.shape[1], y_train.shape[1])
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    return model
def train_model(X_train, y_train):
    model = build_model(X_train.shape[1], y_train.shape[1])
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    return model

def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

# Define the fitness function
def evaluate(individual):
    selected_features = [index for index, value in enumerate(individual) if value == 1]
    if len(selected_features) == 0:
        return 0,

    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    model = train_model(X_train_selected, y_train)
    accuracy = evaluate_model(model, X_test_selected, y_test)
    return accuracy,

# Define GA components
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(feature_columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run the GA
def run_ga():
    population = toolbox.population(n=50)
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, verbose=True)
    return tools.selBest(population, k=1)[0]

best_individual = run_ga()
selected_features = [index for index, value in enumerate(best_individual) if value == 1]


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	50    
1  	29    
2  	33    
3  	33    
4  	36    
5  	33    
6  	30    
7  	33    
8  	35    
9  	24    
10 	26    


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def build_model(input_dim, output_dim):
    model = Sequential([
        Dense(64, input_dim=input_dim, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model using the selected features
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

model = build_model(X_train_selected.shape[1], y_train.shape[1])
model.fit(X_train_selected, y_train, epochs=2000, batch_size=32, validation_data=(X_test_selected, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_selected, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/2000
527/527 [==============================] - 3s 4ms/step - loss: 2.7430 - accuracy: 0.1719 - val_loss: 2.1389 - val_accuracy: 0.3928
Epoch 2/2000
527/527 [==============================] - 2s 4ms/step - loss: 2.2364 - accuracy: 0.3206 - val_loss: 1.7802 - val_accuracy: 0.4909
Epoch 3/2000
527/527 [==============================] - 3s 5ms/step - loss: 1.9967 - accuracy: 0.3818 - val_loss: 1.5785 - val_accuracy: 0.5156
Epoch 4/2000
527/527 [==============================] - 4s 7ms/step - loss: 1.8575 - accuracy: 0.4198 - val_loss: 1.4639 - val_accuracy: 0.5462
Epoch 5/2000
527/527 [==============================] - 2s 4ms/step - loss: 1.7612 - accuracy: 0.4409 - val_loss: 1.3755 - val_accuracy: 0.5759
Epoch 6/2000
527/527 [==============================] - 2s 3ms/step - loss: 1.6968 - accuracy: 0.4551 - val_loss: 1.3074 - val_accuracy: 0.5813
Epoch 7/2000
527/527 [==============================] - 2s 4ms/step - loss: 1.6505 - accuracy: 0.4621 - val_loss: 1.2705 - val_accuracy: